### Imports
### Make sure you add to this cell as you experiement with other types of RNN

In [13]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
import keras
import keras.utils
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import SimpleRNN
from keras.layers import GRU
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
from sklearn.metrics import confusion_matrix

### This section deals with loading the training and testing datasets
### it is not necessary to modify it for the PE10 assignment

In [14]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames):
	loaded = list()
	for name in filenames:
		data = load_file(name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset_train():
	filenames = ['total_acc_x_train.txt','total_acc_y_train.txt','total_acc_z_train.txt','body_acc_x_train.txt','body_acc_y_train.txt','body_acc_z_train.txt','body_gyro_x_train.txt','body_gyro_y_train.txt','body_gyro_z_train.txt']
	# load input data
	X = load_group(filenames)
	# load class output
	y = load_file('y_train.txt')
	return X, y

def load_dataset_test():
  filenames = ['total_acc_x_test.txt','total_acc_y_test.txt','total_acc_z_test.txt','body_acc_x_test.txt','body_acc_y_test.txt','body_acc_z_test.txt','body_gyro_x_test.txt','body_gyro_y_test.txt','body_gyro_z_test.txt']
  # load input data
  X = load_group(filenames)
  # load class output
  y = load_file('y_test.txt')
  return X, y


# load the dataset, returns train and test X and y elements
def load_dataset():
	# load all train
	trainX, trainy = load_dataset_train()
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_test()
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy

In [15]:
# Function to build, compile, and train the model
# For PE10, copy and modify this section to try different model designs and RNNs such as simpleRNN and GRU, in addition to LSTM
# You can try stacking the RNN layers. Refer to the Keras API on how to do this

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  verbose, epochs, batch_size = 0, 15, 64
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  model = Sequential()
  model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
  model.add(Dropout(0.5))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  #model.summary()
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  return accuracy


## LSTM Model Implementation
The following section implements the LSTM-based model for human activity recognition.

In [16]:
# LSTM Model - fit and evaluate
def evaluate_model_lstm(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 15, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

# Run LSTM experiment
def run_lstm_experiment(repeats=10):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_lstm(trainX, trainy, testX, testy)
        score = score * 100.0
        print('LSTM >#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    print('LSTM Results:')
    summarize_results(scores)
    return scores

## SimpleRNN Model Implementation
The following section implements the SimpleRNN-based model for human activity recognition.

In [17]:
# SimpleRNN Model - fit and evaluate
def evaluate_model_simplernn(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 15, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(SimpleRNN(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

# Run SimpleRNN experiment
def run_simplernn_experiment(repeats=10):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_simplernn(trainX, trainy, testX, testy)
        score = score * 100.0
        print('SimpleRNN >#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    print('SimpleRNN Results:')
    summarize_results(scores)
    return scores

## GRU Model Implementation
The following section implements the GRU-based model for human activity recognition.

In [18]:
# GRU Model - fit and evaluate
def evaluate_model_gru(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 15, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(GRU(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

# Run GRU experiment
def run_gru_experiment(repeats=10):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_gru(trainX, trainy, testX, testy)
        score = score * 100.0
        print('GRU >#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    print('GRU Results:')
    summarize_results(scores)
    return scores

### This function summarizes the results of the training

In [19]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

### This is the main code that loads the data and runs the classifier

In [20]:
# run an experiment
def run_experiment(repeats=10):
  # load data
  trainX, trainy, testX, testy = load_dataset()
  # repeat experiment
  scores = list()
  for r in range(repeats):
    score = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)

### Running the code here:
### Make sure you go back and modify the model design when trying out new versions for simpleRNN, LSTM and GRU. 

In [21]:
# Run experiments for all three RNN types
print("="*60)
print("Running LSTM Experiment")
print("="*60)
lstm_scores = run_lstm_experiment(repeats=10)

print("\n" + "="*60)
print("Running SimpleRNN Experiment")
print("="*60)
simplernn_scores = run_simplernn_experiment(repeats=10)

print("\n" + "="*60)
print("Running GRU Experiment")
print("="*60)
gru_scores = run_gru_experiment(repeats=10)

Running LSTM Experiment


/var/folders/hw/mr_tqyws2jx9c63s7wfqf4280000gn/T/ipykernel_98300/1476725414.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = read_csv(filepath, header=None, delim_whitespace=True)
/var/folders/hw/mr_tqyws2jx9c63s7wfqf4280000gn/T/ipykernel_98300/1476725414.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = read_csv(filepath, header=None, delim_whitespace=True)
/var/folders/hw/mr_tqyws2jx9c63s7wfqf4280000gn/T/ipykernel_98300/1476725414.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = read_csv(filepath, header=None, delim_whitespace=True)
/var/folders/hw/mr_tqyws2jx9c63s7wfqf4280000gn/T/ipykernel_98300/1476725414.py:3: FutureWarning: The 'delim_whitespace' k

(7352, 128, 9) (7352, 1)


/var/folders/hw/mr_tqyws2jx9c63s7wfqf4280000gn/T/ipykernel_98300/1476725414.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = read_csv(filepath, header=None, delim_whitespace=True)
/var/folders/hw/mr_tqyws2jx9c63s7wfqf4280000gn/T/ipykernel_98300/1476725414.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = read_csv(filepath, header=None, delim_whitespace=True)
/var/folders/hw/mr_tqyws2jx9c63s7wfqf4280000gn/T/ipykernel_98300/1476725414.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = read_csv(filepath, header=None, delim_whitespace=True)
/var/folders/hw/mr_tqyws2jx9c63s7wfqf4280000gn/T/ipykernel_98300/1476725414.py:3: FutureWarning: The 'delim_whitespace' k

(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


...

(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
SimpleRNN >#1: 73.770
SimpleRNN >#1: 73.770
SimpleRNN >#2: 70.987
SimpleRNN >#2: 70.987
SimpleRNN >#3: 60.943
SimpleRNN >#3: 60.943
SimpleRNN >#4: 88.497
SimpleRNN >#4: 88.497
SimpleRNN >#5: 51.171
SimpleRNN >#5: 51.171
SimpleRNN >#6: 71.293
SimpleRNN >#6: 71.293
SimpleRNN >#7: 40.041
SimpleRNN >#7: 40.041
SimpleRNN >#8: 76.383
SimpleRNN >#8: 76.383
SimpleRNN >#9: 79.199
SimpleRNN >#9: 79.199
SimpleRNN >#10: 67.221
SimpleRNN Results:
[73.76993298530579, 70.98744511604309, 60.94332933425903, 88.49677443504333, 51.17068290710449, 71.29284143447876, 40.040719509124756, 76.38276219367981, 79.19918298721313, 67.22090244293213]
Accuracy: 67.950% (+/-13.357)

Running GRU Experiment
SimpleRNN >#10: 67.221
SimpleRNN Results:
[73.76993298530579, 70.98744511604309, 60.94332933425903, 88.49677443504333, 51.17068290710449, 71.29284143447876, 40.040719509124756, 76.38276219367981, 79.19918298721313, 67.22090244293213]
Accurac

/var/folders/hw/mr_tqyws2jx9c63s7wfqf4280000gn/T/ipykernel_98300/1476725414.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = read_csv(filepath, header=None, delim_whitespace=True)
/var/folders/hw/mr_tqyws2jx9c63s7wfqf4280000gn/T/ipykernel_98300/1476725414.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = read_csv(filepath, header=None, delim_whitespace=True)
/var/folders/hw/mr_tqyws2jx9c63s7wfqf4280000gn/T/ipykernel_98300/1476725414.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = read_csv(filepath, header=None, delim_whitespace=True)
/var/folders/hw/mr_tqyws2jx9c63s7wfqf4280000gn/T/ipykernel_98300/1476725414.py:3: FutureWarning: The 'delim_whitespace' k

(7352, 128, 9) (7352, 1)


/var/folders/hw/mr_tqyws2jx9c63s7wfqf4280000gn/T/ipykernel_98300/1476725414.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = read_csv(filepath, header=None, delim_whitespace=True)
/var/folders/hw/mr_tqyws2jx9c63s7wfqf4280000gn/T/ipykernel_98300/1476725414.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = read_csv(filepath, header=None, delim_whitespace=True)
/var/folders/hw/mr_tqyws2jx9c63s7wfqf4280000gn/T/ipykernel_98300/1476725414.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dataframe = read_csv(filepath, header=None, delim_whitespace=True)
/var/folders/hw/mr_tqyws2jx9c63s7wfqf4280000gn/T/ipykernel_98300/1476725414.py:3: FutureWarning: The 'delim_whitespace' k

(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
GRU >#1: 92.263
GRU >#1: 92.263
GRU >#2: 91.788
GRU >#2: 91.788
GRU >#3: 90.092
GRU >#3: 90.092
GRU >#4: 88.599
GRU >#4: 88.599
GRU >#5: 90.261
GRU >#5: 90.261
GRU >#6: 90.736
GRU >#6: 90.736
GRU >#7: 89.243
GRU >#7: 89.243
GRU >#8: 92.535
GRU >#8: 92.535
GRU >#9: 89.956
GRU >#9: 89.956
GRU >#10: 91.211
GRU Results:
[92.2633171081543, 91.78826212882996, 90.09161591529846, 88.59857320785522, 90.26128053665161, 90.73634147644043, 89.24329876899719, 92.53478050231934, 89.95589017868042, 91.21140241622925]
Accuracy: 90.668% (+/-1.221)
GRU >#10: 91.211
GRU Results:
[92.2633171081543, 91.78826212882996, 90.09161591529846, 88.59857320785522, 90.26128053665161, 90.73634147644043, 89.24329876899719, 92.53478050231934, 89.95589017868042, 91.21140241622925]
Accuracy: 90.668% (+/-1.221)


---
## Summary of Results and Findings

### Quick Answer: Which RNN Works Best?
**🏆 GRU wins with 90.67% accuracy**, followed closely by LSTM at 89.53%. SimpleRNN significantly lags behind at 67.95%.

---

### Detailed Results Comparison

| Model | Average Accuracy | Variation (+/-) | Best Run | Worst Run | Consistency |
|-------|------------------|-----------------|----------|-----------|-------------|
| **GRU** | **90.67%** | **1.22%** | 92.54% | 88.60% | ⭐⭐⭐⭐⭐ Excellent |
| **LSTM** | **89.53%** | **2.05%** | 91.82% | 84.02% | ⭐⭐⭐⭐ Very Good |
| **SimpleRNN** | **67.95%** | **13.36%** | 88.50% | 40.04% | ⭐ Poor |

---

### What the Numbers Tell Us

#### 🥇 GRU (Gated Recurrent Unit) - THE WINNER
- **Average Accuracy**: 90.67%
- **Most Consistent**: Variation of only ±1.22% (very tight range)
- **Performance**: Most reliable and best overall
- **Key Insight**: GRU achieved stable, high performance across all 10 runs

#### 🥈 LSTM (Long Short-Term Memory) - VERY CLOSE SECOND
- **Average Accuracy**: 89.53%
- **Consistency**: Good but slightly more variable (±2.05%)
- **Performance**: Nearly as good as GRU with only 1.14% difference
- **Key Insight**: One run dropped to 84.02%, showing slight instability

#### 🥉 SimpleRNN - SIGNIFICANTLY UNDERPERFORMS
- **Average Accuracy**: 67.95%
- **Highly Inconsistent**: Massive variation of ±13.36% (huge swings)
- **Performance**: Drops as low as 40.04%, reaches 88.50% at best
- **Key Insight**: This model cannot be trusted—results are unpredictable

---

### Why These Results?

#### Why GRU Won 🎯
- **Balanced Architecture**: Has gating mechanisms like LSTM but with fewer parameters
- **Efficient Learning**: Fewer parameters = faster training + better generalization
- **Stable Training**: Gates control information flow smoothly
- **Best of Both Worlds**: Performance of LSTM with efficiency of SimpleRNN

#### Why LSTM Came Close
- **Powerful Memory**: Two gates (input/forget) provide strong control
- **Proven Design**: Well-researched and robust architecture
- **Slight Disadvantage**: More parameters than GRU = occasional overfitting

#### Why SimpleRNN Failed ❌
- **No Gates**: Information either flows fully or gets lost completely
- **Vanishing Gradient Problem**: Cannot learn long-term patterns effectively
- **Training Instability**: Random initialization leads to wildly different results
- **Unreliable**: Can't trust the model—too unpredictable

---

### Practical Interpretation

**If this were a real application:**
- ✅ **Deploy GRU** - You can confidently expect 90% accuracy
- ✅ **Deploy LSTM** - Almost as good, slight risk of lower performance
- ❌ **Don't deploy SimpleRNN** - Risk of getting 40-50% accuracy (terrible!)

---

### Key Takeaway
**GRU is the sweet spot** for this activity recognition task:
- Better than LSTM (90.67% vs 89.53%)
- WAY better than SimpleRNN (90.67% vs 67.95%)
- More consistent (1.22% variation vs 2.05% and 13.36%)
- Faster training (fewer parameters)

This validates GRU's reputation as a modern improvement over LSTM—it achieves comparable or better performance with simpler architecture.

Note: Claude Haiku 4.5 was utilized to assist in completing this problem.